In [3]:
!pip install pandas nltk docx fpdf PyPDF2 langchain_together

  Using cached docx-0.2.4-py3-none-any.whl
  Using cached fpdf-1.7.2.tar.gz (39 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_together-0.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_core-0.3.6-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_openai-0.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached langsmith-0.1.128-py3-none-any.whl.metadata (13 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached orjson-3.10.7-cp312-none-win_amd64.whl.metadata (51 kB)
  Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl.metadata (6.7 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Using cached langchain_together-0.2.0-py3-none-any.whl (12 kB)
Using cached langchain_core-0.3.6-py3-none-any.whl (399 kB)
Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
graphrag 0.3.4 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hsrak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
!pip  python-docx

ERROR: unknown command "python-docx"



In [31]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from fpdf import FPDF
from PyPDF2 import PdfReader
from io import BytesIO
import nltk
import time
import threading
from langchain_together import Together

nltk.download('punkt')

# Define directories
UPLOAD_FOLDER = 'uploads'
PROCESSED_FOLDER = 'processed'
output_directory = "output_pdfs"

# Ensure the directories exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)
os.makedirs(output_directory, exist_ok=True)

# Function to read CSV
def read_csv(file):
    try:
        file_content = file.read()  # Read file content
        df = pd.read_csv(BytesIO(file_content), sep='\t')  # Use BytesIO to pass the content
        return df
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

# Function to read DOCX
# def read_docx(file):
#     try:
#         doc = Document(BytesIO(file.read()))
#         paragraphs = [p.text for p in doc.paragraphs if p.text.strip() != '']
#         return "\n".join(paragraphs)
#     except Exception as e:
#         print(f"Error reading DOCX file: {e}")
#         return None

# Function to read PDF
def read_pdf(file):
    try:
        reader = PdfReader(BytesIO(file.read()))
        text = []
        for page in reader.pages:
            text.append(page.extract_text())
        return "\n".join(filter(None, text))
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return None

# Function to save text to PDF
def save_text_to_pdf(text, pdf_path):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    
    # Add text to the PDF
    for line in text.split('\n'):
        pdf.multi_cell(0, 10, line)
    
    # Save the PDF
    pdf.output(pdf_path)

# Function to divide text into chunks
def divide_into_chunks(text, max_words_per_chunk):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_word_count = 0

    for sentence in sentences:
        words_in_sentence = len(word_tokenize(sentence))
        if current_word_count + words_in_sentence > max_words_per_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_word_count = words_in_sentence
        else:
            current_chunk.append(sentence)
            current_word_count += words_in_sentence

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Summarizer class using Langchain Together
class Summarizer:
    def __init__(self):
        self.llm = Together(
            model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            temperature=0.1,
            max_tokens= 1024,
            top_k=1,
            together_api_key="ce2345c478268c88d8952d1324945d1f5f766bb43ccb0733c954ccebc10c32f3"  
        )
        
    def invoke_with_retry(self, full_input, retries=3, retry_delay=30):
        for attempt in range(retries):
            try:
                output_summary = self.llm.invoke(full_input)
                return output_summary
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt < retries - 1:
                    time.sleep(2 ** attempt)
                else:
                    print("Max retries reached. Could not get a response from the server.")
                    raise e

    def generate_biography(self, input_text):
        prompt = """
       I would like you to generate biography of an interviewee based on the following structured questions in German language and  written in third-person-view, make sure it is in german language only. Note down all the dates and years and Please address each question thoroughly, ensuring that the narrative flows smoothly from one life stage to the next. The biography should include the following information:
       
   Birth and Early Family Life:

When and where was the interviewee born? Include the date and location of birth.
Who are the interviewee's parents? Provide their names, backgrounds, and any relevant details about their lives.
Does the interviewee have any siblings? If so, provide details about them, including names and relationships.
Education:

Which school or schools did the interviewee attend? Mention the date, names of the institutions, locations, and any significant experiences or achievements during their education.
Career and Professional Life:

What profession did the interviewee learn or train for? Mention date and describe the nature of their training or education in this field.
Which jobs or professions has the interviewee practiced? Include details about the dates, roles, companies, or organizations they worked for, and any significant milestones or achievements in their career.
Life Events and Personal Milestones:

What were the formative or significant life events with years mentioned in the interviewee's childhood? Mention dates, Include any experiences that had a lasting impact.
What were the formative or significant life events with years mentioned during the interviewee's adolescence?  Mention dates, Describe how these events influenced their path in life.
What were the formative or significant life events with years mentioned in the interviewee's early adult years? Include details about any dates, transitions, challenges, or accomplishments during this period.
What were the formative or significant life events with years mentioned during the interviewee's adult years?  Mention dates, Describe key experiences that shaped their personal or professional life.
What were the formative or significant life events with years mentioned in the interviewee's late adult years? Highlight any dates, major changes, achievements, or reflections during this time.
Personal Life:

Did the interviewee marry  with years mentioned? If yes, provide details about their spouse,dates, including the name and any significant information about their relationship.
Does the interviewee have children with years mentioned? If so, provide details about their children, dates, including names and any significant life events related to them.
Significant Life Events:

What are the most significant life events that have shaped the interviewee's life with years mentioned? Reflect on how these events with years mentioned impacted their personal growth, relationships, or career, dates.
Please ensure the biography is coherent, chronological, detailed, and presents a well-rounded view of the interviewee's life journey with years mentioned.
        # """

        chunks = divide_into_chunks(input_text, max_words_per_chunk=70000)
        biography_parts = []

        for chunk in chunks:
            full_input = chunk + prompt
            output_summary = self.invoke_with_retry(full_input)
            biography_parts.append(output_summary.strip())

        return " ".join(biography_parts)

    def extend_biography(self, full_biography):
        prompt = f"""
        I would like you to extend this biography to cover missing details in German: ...
        {full_biography}
        """
        
        chunks = divide_into_chunks(full_biography, max_words_per_chunk=70000)
        extended_biography_parts = []
        
        for chunk in chunks:
            full_input = chunk + prompt
            output_summary = self.invoke_with_retry(full_input)
            extended_biography_parts.append(output_summary.strip())

        return " ".join(extended_biography_parts)
    
    def remove_incomplete_sentence(self, biography):
        words = word_tokenize(biography)
        truncated_words = words
        truncated_text = " ".join(truncated_words)
        last_full_stop_index = truncated_text.rfind('.')
        
        if last_full_stop_index != -1:
            return truncated_text[:last_full_stop_index + 1]
        else:
            return truncated_text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hsrak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# # Example to process a file (simulate file input)
# def process_file_simulation(file_path):
#     file_name = os.path.basename(file_path)
    
#     with open(file_path, 'rb') as file:
#         transcript_data = None
#         if file_name.endswith('.csv'):
#             df = read_csv(file)
#             transcript_data = "\n".join(df['Transkript'].fillna('').astype(str))
#             print(transcript_data)
#         elif file_name.endswith('.docx'):
#             transcript_data = read_docx(file)
#         elif file_name.endswith('.pdf'):
#             transcript_data = read_pdf(file)

#         if transcript_data:
#             summarizer = Summarizer()
#             biography = summarizer.generate_biography(transcript_data)
#             print(biography)
#             # extended_biography = summarizer.extend_biography(biography)
#             remove_incomplete = summarizer.remove_incomplete_sentence(biography)
#             print(remove_incomplete)
#             output_pdf_path = os.path.join(PROCESSED_FOLDER, file_name.rsplit('.', 1)[0] + '.pdf')
#             save_text_to_pdf(remove_incomplete, output_pdf_path)
#             print(f"Processed {file_name} and saved to {output_pdf_path}")
#         else:
#             print(f"Failed to process {file_name}")


# process_file_simulation("C:\\Users\\hsrak\\Desktop\\case_study\\Files\\adg0001_er_2024_04_23.csv")

Können wir anfangen?
Also wäre schön, wenn Sie mit Kindheit beginnen würden.
Ich war das erste Enkelkind, einzige Enkelkind, lange Zeit und bin sehr verwöhnt worden, da ich ziemlich viel bei der Großmutter gewesen bin.
Ich bin gebürtig aus Hemer im Sauerland und bin 29.5.25 geboren.
Und, ja, Kindheit verlief eigentlich an und für sich normal, habe allerdings ziemlich unter Migräne zu leiden gehabt schon als Vorschulkind, was sich während der Schulzeit verschlechterte und verschlimmerte.
Ich hätte gerne die höhere Schule besucht, durfte aber nicht, weil mir immer gesagt wurde, ich sei krank.
Die Schulentlassung, also den Hauptschulabschluss, habe ich gemacht 1939.
Und damals war das Pflichtjahr bzw. Haushaltsjahr, eh,  musste gema…, musste gemacht werden, ob man die Schule, äh, besuchte oder bevor man den Beruf ergreifen konnte, so.
Und da meine Großeltern ziemlich, sagen wir mal, einen kleinen Dünkel hatten, durfte ich nicht in einen Einzelhaushalt, wie das üblich war, daher kam ich da

In [ ]:
# # Example to process a file (simulate file input)
# def process_file_simulation(file_path):
#     file_name = os.path.basename(file_path)
    
#     with open(file_path, 'rb') as file:
#         transcript_data = None
#         if file_name.endswith('.csv'):
#             df = read_csv(file)
#             transcript_data = "\n".join(df['Transkript'].fillna('').astype(str))
#             print(transcript_data)
#         elif file_name.endswith('.docx'):
#             transcript_data = read_docx(file)
#         elif file_name.endswith('.pdf'):
#             transcript_data = read_pdf(file)

#         if transcript_data:
#             summarizer = Summarizer()
#             biography = summarizer.generate_biography(transcript_data)
#             print(biography)
#             # extended_biography = summarizer.extend_biography(biography)
#             remove_incomplete = summarizer.remove_incomplete_sentence(biography)
#             print(remove_incomplete)
#             output_pdf_path = os.path.join(PROCESSED_FOLDER, file_name.rsplit('.', 1)[0] + '.pdf')
#             save_text_to_pdf(remove_incomplete, output_pdf_path)
#             print(f"Processed {file_name} and saved to {output_pdf_path}")
#         else:
#             print(f"Failed to process {file_name}")


# process_file_simulation("C:\\Users\\hsrak\\Desktop\\case_study\\Files\\adg0001_er_2024_04_23.csv")

In [36]:
import os

# Example to process a file (simulate file input)
def process_file_simulation(file_path):
    file_name = os.path.basename(file_path)
    
    with open(file_path, 'rb') as file:
        transcript_data = None
        
        # Process based on file type
        if file_name.endswith('.csv'):
            df = read_csv(file)
            transcript_data = "\n".join(df['Transkript'].fillna('').astype(str))
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)  # Print the transcript data
        elif file_name.endswith('.docx'):
            transcript_data = read_docx(file)
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)
        elif file_name.endswith('.pdf'):
            transcript_data = read_pdf(file)
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)

        # If transcript data is successfully extracted, summarize it
        if transcript_data:
            summarizer = Summarizer()
            biography = summarizer.generate_biography(transcript_data)
            print(f"\nGenerated Biography from {file_name}:\n")
            print(biography)  # Print the generated biography
            
            # Optionally, you can also remove incomplete sentences and print the cleaned text
            remove_incomplete = summarizer.remove_incomplete_sentence(biography)
            print(f"\nCleaned Biography (Incomplete sentences removed) from {file_name}:\n")
            print(remove_incomplete)
        else:
            print(f"Failed to process {file_name}")


# Example usage
process_file_simulation("C:\\Users\\hsrak\\Desktop\\case_study\\Files\\adg0001_er_2024_04_23.csv")


Extracted Transcript Data from adg0001_er_2024_04_23.csv:

Können wir anfangen?
Also wäre schön, wenn Sie mit Kindheit beginnen würden.
Ich war das erste Enkelkind, einzige Enkelkind, lange Zeit und bin sehr verwöhnt worden, da ich ziemlich viel bei der Großmutter gewesen bin.
Ich bin gebürtig aus Hemer im Sauerland und bin 29.5.25 geboren.
Und, ja, Kindheit verlief eigentlich an und für sich normal, habe allerdings ziemlich unter Migräne zu leiden gehabt schon als Vorschulkind, was sich während der Schulzeit verschlechterte und verschlimmerte.
Ich hätte gerne die höhere Schule besucht, durfte aber nicht, weil mir immer gesagt wurde, ich sei krank.
Die Schulentlassung, also den Hauptschulabschluss, habe ich gemacht 1939.
Und damals war das Pflichtjahr bzw. Haushaltsjahr, eh,  musste gema…, musste gemacht werden, ob man die Schule, äh, besuchte oder bevor man den Beruf ergreifen konnte, so.
Und da meine Großeltern ziemlich, sagen wir mal, einen kleinen Dünkel hatten, durfte ich nicht in

In [33]:
import os

# Example to process a file (simulate file input)
def process_file_simulation(file_path):
    file_name = os.path.basename(file_path)
    
    with open(file_path, 'rb') as file:
        transcript_data = None
        
        # Process based on file type
        if file_name.endswith('.csv'):
            df = read_csv(file)
            transcript_data = "\n".join(df['Transkript'].fillna('').astype(str))
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)  # Print the transcript data
        elif file_name.endswith('.docx'):
            transcript_data = read_docx(file)
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)
        elif file_name.endswith('.pdf'):
            transcript_data = read_pdf(file)
            print(f"Extracted Transcript Data from {file_name}:\n")
            print(transcript_data)

        # If transcript data is successfully extracted, summarize it
        if transcript_data:
            summarizer = Summarizer()
            biography = summarizer.generate_biography(transcript_data)
            print(f"\nGenerated Biography from {file_name}:\n")
            print(biography)  # Print the generated biography
            
            # Optionally, you can also remove incomplete sentences and print the cleaned text
            remove_incomplete = summarizer.remove_incomplete_sentence(biography)
            print(f"\nCleaned Biography (Incomplete sentences removed) from {file_name}:\n")
            print(remove_incomplete)
        else:
            print(f"Failed to process {file_name}")


# Example usage
process_file_simulation("C:\\Users\\hsrak\\Desktop\\case_study\\adg0002_er_2024_04_23.csv")


Extracted Transcript Data from adg0002_er_2024_04_23.csv:

Johanna Alexander 
Ähm, also ich, äh, erblickte das sogenannte Licht der Welt am 22.12.1918.
Meine Eltern, [kurze Pause] [Klirren] davon war der Vater Weichensteller zu dem Zeitpunkt, meine Mutter war ungelernt.
ja
Hm
Der Altersunterschied zwischen beiden betrug sieben Jahre.
Meine Mutter heiratete mit 19 und bekam kurz hintereinander zwei Kinder.
Der Grund war übrigens auch bereits 'ne Schwangerschaft zur Heirat, und diese beiden Kinder verstarben aber auch, bevor sie ein Jahr alt wurden.
Hm.
Hm
So wurde ich das dritte Kind dieser Eheleute von noch drei nachfolgenden Kindern.
Und ...
Wir waren dann insgesamt vier, die am Leben blieben.
Hm.
Und Sie waren dann praktisch von denen nun, die am Leben blieben, die Älteste?
Ich war von denen nun die Älteste.
Im Abstand von eineinhalb Jahren, zwei Jahren und noch 'mal eineinhalb Jahren wurden dann die Kinder geboren.
Und wie ich schon sagte, mein Vater übte den Beruf eines Weichenstel